In [8]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import datasets, layers, Model
import keras_cv

Using TensorFlow backend


In [4]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

In [5]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [6]:
train_images = train_images.astype(np.float32)
train_images = train_images / 255.0

test_images = test_images.astype(np.float32)
test_images = test_images / 255.0

train_labels = train_labels.astype(np.int32)
test_labels = test_labels.astype(np.int32)

In [ ]:
IMG_SIZE = 32*3

In [ ]:
def image_preprocess(image,label):
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, size=(IMG_SIZE,IMG_SIZE))
    label = tf.cast(label,tf.int32)
    return image,label


randaugment = keras_cv.layers.RandAugment(value_range(0,1))

def image_augmentation(image,label):
    image = randaugment(image)
    return image,label

In [ ]:
batch_size = 128

dataset_train = tf.data.Dataset.from_tensor_slices((train_images,train_labels))
dataset_train = dataset_train.shuffle(buffer_size=len(train_labels))
dataset_train = dataset_train.batch(batch_size)
dataset_train = dataset_train.map(image_preprocess,num_parallel_calls=tf.data.AUTOTUNE)
dataset_train = dataset_train.map(image_augmentation,num_parallel_calls=tf.data.AUTOTUNE)
dataset_train = dataset_train.prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
dataset_test = tf.data.Dataset.from_tensor_slices((test_images,test_labels))
dataset_test = dataset_test.map(image_preprocess,num_parallel_calls=tf.data.AUTOTUNE)
dataset_test = dataset_test.batch(batch_size)
dataset_test = dataset_test.prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
input_shape = (IMG_SIZE,IMG_SIZE,3)

MobileNetV2 = tf.keras.applications.MobileNetV2(
    input_shape=input_shape,
    weights='imagenet',
    include_top=False
)

inputs = MobileNetV2.input

In [ ]:
x = MobileNetV2.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.LayerNormalization()(x)
outputs = layers.Dense(len(class_names),activation='softmax')(x)

In [ ]:
model = Model(inputs=inputs,outputs=outputs)
model.summary(show_trainable=True)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),  # Use 'binary_crossentropy' for binary classification
              metrics=tf.keras.metrics.SparseCategoricalAccuracy())

In [ ]:
history = model.fit(dataset_train,train_labels,epoch=10,validation_data=dataset_test)

In [ ]:
for idx, layer in enumerate(model.layers):
    print('{:<5d}: {}'.format(idx,layer.name))

In [ ]:
for layer in model.layers[:134]:
    layer.trainable = False

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy,  # Use 'binary_crossentropy' for binary classification
              metrics=tf.keras.SparseCategoricalAccuracy())

In [ ]:
model.summary(show_trainable=True)

In [ ]:
history_fine = model.fit(dataset_train,train_labels,epoch=10,validation_data=(dataset_test,test_labels))
